# **Composer classification Model**
___

#### **Problem Statement**

We are going to be working with the MIDI files of classical music composers. We will be using the MIDI files to classify the composer of the music. We will be using the MIDI files of the following composers: Bach Beethoven, chopin and Mozart. We will be building a CNN network to classify the composers, and we will also be making functions that will make the process of dealing with the midi files easier to work with. The objective is to get the classification model to be at approximately 90% accuracy, or better. 

In [13]:
# librarys for the model and working with data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from concurrent.futures import ProcessPoolExecutor

# deep learning libraries for the CNN classification model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping

# working with the midi files
import pretty_midi
import librosa.display
import mido

In [14]:
# Define the path to the MIDI files
main_dir = 'midiclassics'

In [15]:
# Function to extract features from MIDI files
def extract_midi_features(file_path, max_sequence_length=300):
    try:
        midi_data = pretty_midi.PrettyMIDI(file_path)
        tempo = midi_data.estimate_tempo()
        key_signatures = [key.key_number for key in midi_data.key_signature_changes]
        time_signatures = [(time.numerator, time.denominator) for time in midi_data.time_signature_changes]
        instrument_types = [instr.program for instr in midi_data.instruments]
        notes_histogram = midi_data.get_pitch_class_histogram()
        notes = np.zeros((max_sequence_length, 128))
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                start = int(note.start * max_sequence_length / midi_data.get_end_time())
                end = int(note.end * max_sequence_length / midi_data.get_end_time())
                notes[start:end, note.pitch] = note.velocity / 127
        return {
            'tempo': tempo,
            'key_signatures': key_signatures,
            'time_signatures': time_signatures,
            'instrument_types': instrument_types,
            'notes_histogram': notes_histogram.tolist(),
            'notes': notes
        }
    except Exception as e:
        print(f"Failed to process {file_path}: {e}")
        return None

In [16]:
# Function to get MIDI data from the main directory
def get_midi_data(main_dir, max_sequence_length=300):
    composers_data = {}
    
    # Function to process a single composer's folder
    def process_composer_folder(folder):
        folder_path = os.path.join(main_dir, folder)
        data = []
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if file_path.endswith('.midi') or file_path.endswith('.mid') or file_path.endswith('.MID'):
                features = extract_midi_features(file_path, max_sequence_length)
                if features is not None:
                    features['file'] = file
                    data.append(features)
        return folder, pd.DataFrame(data)
    
    # Use ProcessPoolExecutor for parallel processing
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_composer_folder, folder) for folder in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, folder))]
        for future in futures:
            folder, df = future.result()
            composers_data[folder] = df
    
    return composers_data


In [17]:
# Load MIDI data
all_composers_data = get_midi_data(main_dir)

In [ ]:
# Data preprocessing
def prepare_data(df, composer_name, max_sequence_length=300):
    df['composer'] = composer_name
    scaler = StandardScaler()
    df['tempo'] = scaler.fit_transform(df[['tempo']])
    df['padded_notes'] = pad_sequences(df['notes'].tolist(), maxlen=max_sequence_length, padding='post', dtype='float32').tolist()
    return df

In [ ]:
# Combine and split dataset
def prepare_features_and_labels(combined_data):
    X = np.array(combined_data['padded_notes'].tolist())
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(combined_data['composer'])
    y_categorical = to_categorical(y)
    return X, y_categorical, label_encoder.classes_

In [ ]:
# inital model
def create_initial_model(input_shape, n_classes, initial_units=64):
    model = Sequential([
        LSTM(initial_units, input_shape=input_shape, return_sequences=True),
        Dropout(0.5),
        LSTM(initial_units // 2, return_sequences=True),
        Dropout(0.5),
        LSTM(initial_units // 4, return_sequences=False),
        Dropout(0.5),
        Dense(n_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# custom training loop
def progressive_training(X_train, y_train, X_test, y_test, initial_units=64, max_layers=10, peak_layers=6):
    input_shape = (X_train.shape[1], X_train.shape[2])
    n_classes = y_train.shape[1]
    model = create_initial_model(input_shape, n_classes, initial_units)

    best_accuracy = 0
    current_layer_count = 3  # Starting with 3 LSTM layers

    for i in range(max_layers):
        checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='max')

        model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test),
                  callbacks=[checkpoint, early_stopping])
        model.load_weights('best_model.h5')
        loss, accuracy = model.evaluate(X_test, y_test)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            if best_accuracy >= 0.90:
                print("Reached 90% accuracy. Stopping training...")
                break

        if current_layer_count < peak_layers:
            model.pop()  # Remove softmax layer
            model.add(LSTM(initial_units // (2 ** (current_layer_count - 2)), return_sequences=(current_layer_count != peak_layers)))
            model.add(Dropout(0.5))
            if current_layer_count == peak_layers:
                model.add(Dense(n_classes, activation='softmax'))
            current_layer_count += 1
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        else:
            print("Reached maximum layers. Stopping training...")
            break

    return model

In [ ]:
# Load MIDI data
all_composers_data = get_midi_data(main_dir)


c:\Users\tehwh\anaconda3\envs\ml\lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Failed to process midiclassics\Beethoven\Anhang 14-3.mid: Could not decode key with 3 flats and mode 255


KeyboardInterrupt: 

In [ ]:
# Prepare data
prepared_dfs = []
for composer_name, df in all_composers_data.items():
    prepared_dfs.append(prepare_data(df, composer_name, 300))
combined_data = pd.concat(prepared_dfs, ignore_index=True)


In [ ]:
# split data 
X, y, classes = prepare_features_and_labels(combined_data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# train 
model = progressive_training(X_train, y_train, X_test, y_test)